Birliktelik analizini kullanarak lokasyon ve segmentler için birliktelik kuralları üretip bu kurallara göre öneriler yapma

- Germany (2010-2011)
- cltv_p segmenleri için.

Kritik nokta kuralların tüm veriden ve her segmentin kendi içinden öğrenilmesi gerekli.  
Fakat önerilerin ise ülke özelinde ve yine segment özelinde olması beklenmektedir.

In [1]:
import datetime as dt
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from mlxtend.frequent_patterns import apriori, association_rules
from helpers import *

In [2]:
pd.set_option('display.max_columns', None)
df_ = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")
df = df_.copy()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
# Verinin db'den alınması.

# credentials.
creds = {'user': 'oz',
         'passwd': 'sorryicantsharewithu',
         'host': 'db.github.rocks',
         'port': 3306,
         'db': 'dsmlbc'}

In [ ]:
# credentials.
creds = {'user': 'synan',
         'passwd': 'haydegidelum',
         'host': 'db.github.rocks',
         'port': 3306,
         'db': 'dsmlbc'}

In [ ]:
# MySQL conection string.
connstr = 'mysql+mysqlconnector://{user}:{passwd}@{host}:{port}/{db}'
# sqlalchemy engine for MySQL connection.
conn = create_engine(connstr.format(**creds))

In [ ]:
query = "show databases"
pd.read_sql_query(query, conn)

In [ ]:
query = "select * from online_retail_2010_2011 limit 5"
pd.read_sql_query(query, conn)

In [ ]:
query = "select * from online_retail_2010_2011"
df = pd.read_sql_query(query, conn)

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# Data Preprocessing
df_prep = crm_data_prep(df)
check_df(df_prep)

##################### Shape #####################
(397925, 9)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object
##################### Head #####################
  Invoice StockCode                         Description  Quantity  \
0  536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER       6.0   
1  536365     71053                 WHITE METAL LANTERN       6.0   
2  536365    84406B      CREAM CUPID HEARTS COAT HANGER       8.0   

          InvoiceDate  Price  Customer ID         Country  TotalPrice  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom       15.30  
1 2010-12-01 08:26:00   3.39      17850.0  United Kingdom       20.34  
2 2010-12-01 08:26:00   2.75      17850.0  United Kingdom     

### create_cltv_p Fonksiyonu ile Predictive CLTV Segmentlerini Oluşturma

In [6]:
def create_cltv_p(dataframe):
    today_date = dt.datetime(2011, 12, 11)

    ## recency kullanıcıya özel dinamik.
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max()-date.min()).days,
                                                                lambda date: (today_date - date.min()).days],
                                                'Invoice': lambda num: num.nunique(),
                                                'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

    rfm.columns = rfm.columns.droplevel(0)

    ## recency_cltv_p
    rfm.columns = ['recency_cltv_p', 'T', 'frequency', 'monetary']

    ## basitleştirilmiş monetary_avg
    rfm["monetary"] = rfm["monetary"] / rfm["frequency"]

    rfm.rename(columns={"monetary": "monetary_avg"}, inplace=True)


    # BGNBD için WEEKLY RECENCY VE WEEKLY T'nin HESAPLANMASI
    ## recency_weekly_cltv_p
    rfm["recency_weekly_cltv_p"] = rfm["recency_cltv_p"] / 7
    rfm["T_weekly"] = rfm["T"] / 7



    # KONTROL
    rfm = rfm[rfm["monetary_avg"] > 0]

    ## recency filtre (daha saglıklı cltvp hesabı için)
    rfm = rfm[(rfm['frequency'] > 1)]

    rfm["frequency"] = rfm["frequency"].astype(int)

    # BGNBD
    bgf = BetaGeoFitter(penalizer_coef=0.01)
    bgf.fit(rfm['frequency'],
            rfm['recency_weekly_cltv_p'],
            rfm['T_weekly'])

    # exp_sales_1_month
    rfm["exp_sales_1_month"] = bgf.predict(4,
                                           rfm['frequency'],
                                           rfm['recency_weekly_cltv_p'],
                                           rfm['T_weekly'])
    # exp_sales_3_month
    rfm["exp_sales_3_month"] = bgf.predict(12,
                                           rfm['frequency'],
                                           rfm['recency_weekly_cltv_p'],
                                           rfm['T_weekly'])

    # expected_average_profit
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(rfm['frequency'], rfm['monetary_avg'])
    rfm["expected_average_profit"] = ggf.conditional_expected_average_profit(rfm['frequency'],
                                                                             rfm['monetary_avg'])
    # 6 aylık cltv_p
    cltv = ggf.customer_lifetime_value(bgf,
                                       rfm['frequency'],
                                       rfm['recency_weekly_cltv_p'],
                                       rfm['T_weekly'],
                                       rfm['monetary_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)

    rfm["cltv_p"] = cltv

    # minmaxscaler
    scaler = MinMaxScaler(feature_range=(1, 100))
    scaler.fit(rfm[["cltv_p"]])
    rfm["cltv_p"] = scaler.transform(rfm[["cltv_p"]])

    # rfm.fillna(0, inplace=True)

    # cltv_p_segment
    rfm["cltv_p_segment"] = pd.qcut(rfm["cltv_p"], 3, labels=["C", "B", "A"])

    ## recency_cltv_p, recency_weekly_cltv_p
    rfm = rfm[["recency_cltv_p", "T", "monetary_avg", "recency_weekly_cltv_p", "T_weekly",
               "exp_sales_1_month", "exp_sales_3_month", "expected_average_profit",
               "cltv_p", "cltv_p_segment"]]


    return rfm

In [7]:
cltv_p = create_cltv_p(df_prep)

In [8]:
check_df(cltv_p)

##################### Shape #####################
(2845, 10)
##################### Types #####################
recency_cltv_p                int64
T                             int64
monetary_avg                float64
recency_weekly_cltv_p       float64
T_weekly                    float64
exp_sales_1_month           float64
exp_sales_3_month           float64
expected_average_profit     float64
cltv_p                      float64
cltv_p_segment             category
dtype: object
##################### Head #####################
             recency_cltv_p    T  monetary_avg  recency_weekly_cltv_p  \
Customer ID                                                             
12347.0                 365  368    615.714286              52.142857   
12348.0                 282  359    442.695000              40.285714   
12352.0                 260  297    219.542500              37.142857   

              T_weekly  exp_sales_1_month  exp_sales_3_month  \
Customer ID                         

In [9]:
cltv_p.head()

,recency_cltv_p,T,monetary_avg,recency_weekly_cltv_p,T_weekly,exp_sales_1_month,exp_sales_3_month,expected_average_profit,cltv_p,cltv_p_segment
Customer ID,,,,,,,,,,
12347.0,365,368,615.714286,52.142857,52.571429,0.561684,1.678069,631.911974,2.933517,A
12348.0,282,359,442.695000,40.285714,51.285714,0.364322,1.087951,463.745539,1.919391,B
12352.0,260,297,219.542500,37.142857,42.428571,0.739160,2.206857,224.886669,1.904146,B
12356.0,302,326,937.143333,43.142857,46.571429,0.333040,0.993934,995.997679,2.802492,A
12358.0,149,151,575.210000,21.285714,21.571429,0.474048,1.408955,631.900951,2.612984,A


In [10]:
cltv_p.groupby("cltv_p_segment").agg({"count", "mean"})

recency_cltv_p                 T       monetary_avg        \
                         mean count        mean count         mean count   
cltv_p_segment                                                             
C                  169.920969   949  280.121180   949   198.315161   949   
B                  213.163502   948  257.175105   948   317.029688   948   
A                  213.662447   948  239.033755   948   601.261390   948   

               recency_weekly_cltv_p         T_weekly       exp_sales_1_month  \
                                mean count       mean count              mean   
cltv_p_segment                                                                  
C                          24.274424   949  40.017311   949          0.323827   
B                          30.451929   948  36.739301   948          0.566315   
A                          30.523207   948  34.147679   948          1.027486   

                     exp_sales_3_month       expected_average_profit        \
               count              mean count                    mean count   
cltv_p_segment                                                               
C                949          0.964801   949              214.541431   949   
B                948          1.686891   948              335.687124   948   
A                948          3.061011   948              626.565366   948   

                  cltv_p        
                    mean count  
cltv_p_segment                  
C               1.318969   949  
B               1.899350   948  
A               4.311039   948

### İstenilen segmentlere ait kullanıcı id'lerine göre veri setini indirgeme

In [11]:
# id'lerin alınması
a_segment_ids = cltv_p[cltv_p["cltv_p_segment"] == "A"].index
b_segment_ids = cltv_p[cltv_p["cltv_p_segment"] == "B"].index
c_segment_ids = cltv_p[cltv_p["cltv_p_segment"] == "C"].index

In [12]:
# bu id'lere göre df'lerin indirgenmesi
a_segment_df = df_prep[df_prep["Customer ID"].isin(a_segment_ids)]
b_segment_df = df_prep[df_prep["Customer ID"].isin(b_segment_ids)]
c_segment_df = df_prep[df_prep["Customer ID"].isin(c_segment_ids)]
a_segment_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
9,536368,22960,JAM MAKING SET WITH JARS,6.0,2010-12-01 08:34:00,4.25,13047.0,United Kingdom,25.50
10,536368,22913,RED COAT RACK PARIS FASHION,3.0,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
11,536368,22912,YELLOW COAT RACK PARIS FASHION,3.0,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
12,536368,22914,BLUE COAT RACK PARIS FASHION,3.0,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
13,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32.0,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08


### Her bir segment için birliktelik kurallarının üretilmesi

In [13]:
def create_rules(dataframe, country=False, head=5):
    if country:
        dataframe = dataframe[dataframe['Country'] == country]
        dataframe = create_invoice_product_df(dataframe)
        frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True, low_memory=True)
        rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
        print(rules.sort_values("lift", ascending=False).head(head))
    else:
        dataframe = create_invoice_product_df(dataframe)
        frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True, low_memory=True)
        rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
        print(rules.sort_values("lift", ascending=False).head(head))

    return rules

In [14]:
rules_a = create_rules(a_segment_df)
product_a = int(rules_a["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0])

rules_b = create_rules(b_segment_df)
product_b = int(rules_b["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0])

rules_c = create_rules(c_segment_df)
product_c = int(rules_c["consequents"].apply(lambda x: list(x)[0]).astype("unicode")[0])

     antecedents consequents  antecedent support  consequent support  \
1026     (22920)     (22916)            0.011129            0.010814   
1027     (22916)     (22920)            0.010814            0.011129   
1028     (22917)     (22918)            0.011549            0.010814   
1029     (22918)     (22917)            0.010814            0.011549   
1033     (22917)     (22920)            0.011549            0.011129   

       support  confidence       lift  leverage  conviction  
1026  0.010184    0.915094  84.624015  0.010063   11.650417  
1027  0.010184    0.941748  84.624015  0.010063   16.975626  
1028  0.010499    0.909091  84.068844  0.010374   10.881050  
1029  0.010499    0.970874  84.068844  0.010374   33.936833  
1033  0.010709    0.927273  83.323328  0.010580   13.596982  
    antecedents consequents  antecedent support  consequent support   support  \
481     (22917)     (22916)            0.011040            0.010819  0.010378   
480     (22916)     (22917)      

In [15]:
def check_id(stock_code):
    product_name = df_prep[df_prep["StockCode"] == stock_code][["Description"]].values[0].tolist()
    return print(product_name)

In [16]:
check_id(20719)

['WOODLAND CHARLOTTE BAG']
